### <h>Twitter Tweet Analysis</h>

In [657]:
### Import Packages

import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import tweepy
from tweepy import OAuthHandler #twitter app authentication
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.stem import PorterStemmer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [658]:
# Get the below strings/api codes from developer.twitter.com
# Step 1: Login to twitter and visit the developer site
# Step 2 : Navigate to "Create a new App"
# Step 3 : fill the details accordingly
# Step 4 : api keys are now generated
# keys and tokens from the Twitter Dev Console 


consumer_key = 'nT6RnPI3pE76JGmRi2cyqkBMW'
consumer_secret = 'Lj5SP5hhonyRbEebVTdHPWg6hws23wS9VI3yPxAlOWOZ2i669M'
access_token = '141093387-Aa6UKvi6kqLZqrwvOesZumvifHrYKfARNfkpNptb'
access_token_secret = 'Ielj2fpP1ldhhY74uk2mLTvz7roSskXqRciomc1Keh6t2'

In [659]:
##Twiiter Authentication

# create OAuthHandler object 
auth = OAuthHandler(consumer_key,consumer_secret)

# set access token and secret
auth.set_access_token(access_token,access_token_secret)

# create tweepy API object to fetch tweets
api = tweepy.API(auth,wait_on_rate_limit=True) 

### step 1: Text Cleaning  & Analyse

In [660]:
# 1. Remove @ Username
# 2. tabs
# 3. Non words and spaces
# 4. Clean Symbols
# 5. Lower Text


def Clean_Tweet(tweet):
    
    return " ".join(re.sub('(@[A-Za-z0-9]+)|([^A-Za-z0-9 \t])|(\w+:\/\/\S+)|([RT])','',tweet).split())

In [661]:
def Clean_Symbols(tweet):
    Clean_Data = []
    for val in tweet:
        if (val.isalnum() or val == " "):
            Clean_Data.append(val)
    return "".join(Clean_Data)

In [662]:
## Get Sentiment Anlysis

def get_tweet_sentiment(tweet):
    
    analysis = TextBlob(tweet)
    
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Not Defined'

In [663]:
def polarity_tweet(tweet):
    
    tb = TextBlob(tweet)
    polarity_factor = tb.sentiment.polarity
    
    return polarity_factor

In [ ]:
tweets = []
fetched_tweets=tweepy.Cursor(api.search,q="#SpeakUpForSSCRailwayStudents").items(1000)

for tweet in fetched_tweets:
    
    parsed_tweets = {}
    parsed_tweets['text'] = tweet.text
    #parsed_tweets['sentiment'] = get_tweet_sentiment(tweet=Clean_Tweet(tweet.text))
    #parsed_tweets['polarity'] = polarity_tweet(tweet=Clean_Tweet(tweet.text))
    tweets.append(parsed_tweets)
    

In [ ]:
tweets[0:5]

In [ ]:
###Inserting Data into DataFrame for CleeanUp & further processing

tw_text = []

for x in tweets:
    tw_text.append(x['text'])
    
tw_text = pd.Series(tw_text, name = "tweet_text")
sentiment_analysis = pd.DataFrame(tw_text)

In [ ]:
sentiment_analysis

In [ ]:
##Clean Text using Regex Function

for x in sentiment_analysis.index:
    sentiment_analysis.tweet_text[x] = Clean_Tweet(sentiment_analysis.tweet_text[x])

In [ ]:
### Lower Case the Test Data and modify incorrect words

for x in sentiment_analysis.index:
    sentiment_analysis.tweet_text[x] = sentiment_analysis.tweet_text[x].lower()

In [ ]:
##Remove Stop Words

stop = set(nltk.corpus.stopwords.words('english'))
sentiment_analysis.tweet_text = sentiment_analysis['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
sentiment_analysis

In [ ]:
sentiment_analysis['Polarity'] = sentiment_analysis['tweet_text'].apply(lambda x:polarity_tweet(x))

In [ ]:
sentiment_analysis['sentiment'] = sentiment_analysis['tweet_text'].apply(lambda x:get_tweet_sentiment(x))

In [ ]:
posive_tweets = [x for x in sentiment_analysis.sentiment if x == 'Positive']
print("Positive tweets percentage: {} %".format(100*len(posive_tweets)/len(tweets))) 

In [ ]:
Negative_tweets = [x for x in sentiment_analysis.sentiment if x == 'Negative']
print("Neagtive tweets percentage: {} %".format(100*len(Negative_tweets)/len(tweets))) 

In [ ]:
Neutral_tweets = [x for x in  sentiment_analysis.sentiment if x == 'Neutral']
print("Neutral tweets percentage: {} %".format(100*len(Neutral_tweets)/len(tweets))) 

#### Tweet Count

In [ ]:
tweet_count = pd.DataFrame(pd.concat([pd.Series(len(posive_tweets)),pd.Series(len(Negative_tweets)),pd.Series(len(Neutral_tweets))],axis=0))
tweet_count.set_index(pd.Series(['Positive_Tweets','Negative_Tweets','Neutral_Tweets']),inplace = True)
print(tweet_count)

In [ ]:
class_data.Sentiment.value_counts()

In [ ]:
tweet_count.plot(kind='bar')

In [ ]:
len(sentiment_analysis)

In [ ]:
## Correcting Hindi/Any other words into English
